In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import operator


teamsSinceSaison = 2018 - 20
trainerSinceSaison = 2017
trainerTournament = "1. Bundesliga"

untilSaison = 2019

In [2]:
!ls /goalimpacct/data_compressed/

card           match          player-profile team           trainer-name
goal           player         substitution   trainer


In [3]:
TRAINER_RAW = "/goalimpacct/data_compressed/trainer/compressed.csv"
rawTrainerDF =  pd.read_csv(TRAINER_RAW).drop_duplicates()
rawTrainerDF.shape

(164688, 3)

In [4]:
TRAINER_NAME_RAW = "/goalimpacct/data_compressed/trainer-name/compressed.csv"
rawTrainerNameDF =  pd.read_csv(TRAINER_NAME_RAW).drop_duplicates()
rawTrainerNameDF.shape

(7671, 2)

In [5]:
MATCH_CSV = "/goalimpacct/data_compressed/match/compressed.csv"
rawMatchDF =  pd.read_csv(MATCH_CSV)
rawMatchDF.shape

(80412, 7)

In [6]:
GOAL_CSV = "/goalimpacct/data_compressed/goal/compressed.csv"
rawGoalDF =  pd.read_csv(GOAL_CSV)
rawGoalDF.shape

(229884, 6)

In [7]:
rawMatchDF

,:id,:tournament,:date,:home,:guest,:saison,:match-length
0,1,UEFA Champions League,17.09.2003,27.0,371.0,2003,90.0
1,10,UEFA Champions League,16.09.2003,985.0,265.0,2003,90.0
2,1000004,Scottish League One,20.03.2010,3032.0,3021.0,2009,90.0
3,1000005,Scottish League One,27.03.2010,3021.0,3020.0,2009,90.0
4,1000011,Scottish League One,03.04.2010,2575.0,3021.0,2009,90.0
5,1000021,Scottish League One,17.04.2010,3021.0,3027.0,2009,90.0
6,1000026,Scottish League One,24.04.2010,3023.0,3021.0,2009,90.0
7,1000064,ÖFB-Cup,18.09.2001,7001.0,122.0,2001,90.0
8,1000076,ÖFB-Cup,19.03.2002,170.0,409.0,2001,90.0
9,1000095,LaLiga,17.11.2002,150.0,940.0,2002,90.0


In [8]:
goalsPerMatch = rawGoalDF.groupby([':team', ':match']).size().reset_index(name='counts')

In [9]:
homeGoals = goalsPerMatch.rename(columns={'counts': 'homeGoals'})
homeMatchDF = rawMatchDF.merge(homeGoals, left_on=[':home', ':id'], right_on=[':team', ':match']).drop(columns=[':team', ':match'])

In [10]:
guestGoals = goalsPerMatch.rename(columns={'counts': 'guestGoals'})
goalMatchDF = homeMatchDF.merge(guestGoals, left_on=[':guest', ':id'], right_on=[':team', ':match']).drop(columns=[':team', ':match'])

In [11]:
goalMatchDF

,:id,:tournament,:date,:home,:guest,:saison,:match-length,homeGoals,guestGoals
0,1,UEFA Champions League,17.09.2003,27.0,371.0,2003,90.0,2,1
1,1000021,Scottish League One,17.04.2010,3021.0,3027.0,2009,90.0,1,1
2,1000064,ÖFB-Cup,18.09.2001,7001.0,122.0,2001,90.0,1,6
3,1000076,ÖFB-Cup,19.03.2002,170.0,409.0,2001,90.0,1,2
4,1000095,LaLiga,17.11.2002,150.0,940.0,2002,90.0,2,1
5,1000123,LaLiga,08.12.2002,714.0,150.0,2002,90.0,2,4
6,1000144,LaLiga,22.12.2002,150.0,331.0,2002,90.0,2,1
7,1000163,LaLiga,12.01.2003,150.0,1108.0,2002,90.0,2,2
8,1000181,LaLiga,26.01.2003,150.0,1050.0,2002,90.0,2,1
9,1000199,Allsvenskan,11.04.2010,1011.0,2719.0,2009,90.0,1,2


In [12]:
homeScoresDF = goalMatchDF.drop(columns=[ ':date', ':guest', ':match-length']).rename(columns={':home': ':team'})
homeScoresDF['score'] = np.where(homeScoresDF['homeGoals'] > homeScoresDF['guestGoals'], 3, np.where(homeScoresDF['homeGoals'] < homeScoresDF['guestGoals'], 0, 1))
homeScoresDF.drop(columns=['homeGoals', 'guestGoals'], inplace=True)


In [13]:
guestScoresDF = goalMatchDF.drop(columns=[':date', ':home',':match-length']).rename(columns={':guest': ':team'})
guestScoresDF['score'] = np.where(guestScoresDF['guestGoals'] > guestScoresDF['homeGoals'], 3, np.where(guestScoresDF['guestGoals'] < guestScoresDF['homeGoals'], 0, 1))
guestScoresDF.drop(columns=['guestGoals', 'homeGoals'], inplace=True)


In [14]:
teamScoresDF = pd.concat([homeScoresDF, guestScoresDF]).rename(columns={":id":":match"})

teamScoresDF = teamScoresDF[teamScoresDF[':saison'] >= teamsSinceSaison]
teamScoresDF = teamScoresDF[teamScoresDF[':saison'] < untilSaison]

In [15]:
teamScoresDF

,:match,:tournament,:team,:saison,score
0,1,UEFA Champions League,27.0,2003,3
1,1000021,Scottish League One,3021.0,2009,1
2,1000064,ÖFB-Cup,7001.0,2001,0
3,1000076,ÖFB-Cup,170.0,2001,0
4,1000095,LaLiga,150.0,2002,3
5,1000123,LaLiga,714.0,2002,0
6,1000144,LaLiga,150.0,2002,3
7,1000163,LaLiga,150.0,2002,1
8,1000181,LaLiga,150.0,2002,3
9,1000199,Allsvenskan,1011.0,2009,0


In [16]:


matchesSince = teamScoresDF.loc[
    operator.and_(teamScoresDF[":saison"] >= trainerSinceSaison,
    teamScoresDF[":tournament"] == trainerTournament)
]


relevantTeams = matchesSince[':team'].drop_duplicates()
relevantTeamScoresDF = teamScoresDF.merge(relevantTeams)

relevantTeamScoresDF = relevantTeamScoresDF.loc[relevantTeamScoresDF[":saison"] >= trainerSinceSaison]

In [17]:
relevantTeamScoresDF

,:match,:tournament,:team,:saison,score


In [18]:
trainerScoresDF = relevantTeamScoresDF.merge(rawTrainerDF, on=[':team', ':match']).merge(rawTrainerNameDF, on=":trainer")

In [19]:
scoresPerTrainerAndTeam = trainerScoresDF.groupby([':trainer', ':team', ':trainername'])['score'].agg([
    'mean', 'count']).reset_index()

scoresPerTrainerAndTeam.rename(columns={"mean":"trainerMean", "count" : "trainerMatchCount"}, inplace=True)


In [20]:
scoresPerTeam = teamScoresDF.groupby([':team'])['score'].agg([
    'mean', 'count']).reset_index()

scoresPerTeam.rename(columns={"mean":"teamMean", "count" : "teamMatchCount"}, inplace=True)


In [21]:
diffScoresTeamTrainer = scoresPerTeam.merge(scoresPerTrainerAndTeam, on=":team")

diffScoresTeamTrainer["trainerTeamDiff"] = diffScoresTeamTrainer["trainerMean"] - diffScoresTeamTrainer["teamMean"]

diffScoresTeamTrainer

,:team,teamMean,teamMatchCount,:trainer,:trainername,trainerMean,trainerMatchCount,trainerTeamDiff


In [22]:
trainerDiff = diffScoresTeamTrainer.groupby([
    ':trainer', ':trainername', ':team', 'teamMean'])['trainerTeamDiff', 'trainerMatchCount'].agg([
    'mean', 'sum']).reset_index()

trainerDiff

Empty DataFrame
Columns: [(:trainer, ), (:trainername, ), (:team, ), (teamMean, ), (trainerTeamDiff, mean), (trainerTeamDiff, sum), (trainerMatchCount, mean), (trainerMatchCount, sum)]
Index: []

In [23]:
trainerDiff['trainerTeamDiffs'] = trainerDiff['trainerTeamDiff']['mean']
trainerDiff['trainerMatchCounts'] = trainerDiff['trainerMatchCount']['sum']
trainerDiff.drop(columns=['trainerTeamDiff', 'trainerMatchCount'], inplace=True)

/usr/local/lib/python3.7/site-packages/pandas/core/generic.py:3812: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  new_axis = axis.drop(labels, errors=errors)


In [24]:
trainerSince = matchesSince.merge(rawTrainerDF, on=":match")[':trainer'].drop_duplicates()


In [25]:
trainerDiff.merge(trainerSince, left_on=":trainer", 
                  right_on=":trainer").sort_values(by=['trainerTeamDiffs'], ascending=False)

/usr/local/lib/python3.7/site-packages/pandas/core/reshape/merge.py:522: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


KeyError: 'trainerTeamDiffs'